In [1]:
import pandas as pd
import numpy as np
import re

Importing data in pandas

In [2]:
from pandas.io.pytables import DataCol
data = pd.read_excel('/content/drive/MyDrive/zebra classification /Zebras.Assumption.data_Bing_413 .xlsx',
              sheet_name='Assumptions.all')

data  

,file,q25,q50,q75,fpeak,duration,am.var,am.rate,am.extent,harmonicity
0,05011.squeal.trim.wav,1.024,22.542,144.346,509.753,0.234,203.701,25.684,7.931,10.16
1,05073.squeal.trim.wav,4.626,391.921,635.612,537.097,0.155,204.375,16.133,12.668,20.27
2,05074.squeal.trim.wav,9.773,558.276,806.924,769.535,0.087,243.896,5.745,42.452,17.58
3,18023.squeal.trim.wav,378.288,439.051,489.999,442.747,0.165,245.843,9.098,27.022,13.51
4,19002.squeal.trim.wav,414.270,485.271,627.478,449.798,0.138,225.379,18.142,12.423,13.93
...,...,...,...,...,...,...,...,...,...,...
408,83033.snort.trim.wav,2.413,33.731,162.388,689.083,0.303,413.913,34.701,11.928,6.15
409,83035.snort.trim.wav,12.324,106.265,286.088,181.432,0.169,386.167,29.581,13.054,8.15
410,83051.snort.trim.wav,1.511,47.826,302.461,197.067,0.549,393.041,25.510,15.407,4.52
411,83053.snort.trim.wav,4.978,70.706,211.919,94.333,0.669,477.292,29.899,15.963,7.78


Creating the labels from the 'file' column

In [3]:
labels = []
for i in data['file']:
  y = re.findall('whinnie|snort|softsnort|squeal', i)
  labels.append(y)


In [ ]:
#clever way of doing that above
# [re.findall('whinnie|snort|softsnort|squeal', i) for i in data['file']]

Adding the labels to the dataset

In [4]:
data[['labels']] = labels

Removing the file column

In [5]:
data.drop("file", axis=1, inplace = True)
data

,q25,q50,q75,fpeak,duration,am.var,am.rate,am.extent,harmonicity,labels
0,1.024,22.542,144.346,509.753,0.234,203.701,25.684,7.931,10.16,squeal
1,4.626,391.921,635.612,537.097,0.155,204.375,16.133,12.668,20.27,squeal
2,9.773,558.276,806.924,769.535,0.087,243.896,5.745,42.452,17.58,squeal
3,378.288,439.051,489.999,442.747,0.165,245.843,9.098,27.022,13.51,squeal
4,414.270,485.271,627.478,449.798,0.138,225.379,18.142,12.423,13.93,squeal
...,...,...,...,...,...,...,...,...,...,...
408,2.413,33.731,162.388,689.083,0.303,413.913,34.701,11.928,6.15,snort
409,12.324,106.265,286.088,181.432,0.169,386.167,29.581,13.054,8.15,snort
410,1.511,47.826,302.461,197.067,0.549,393.041,25.510,15.407,4.52,snort
411,4.978,70.706,211.919,94.333,0.669,477.292,29.899,15.963,7.78,snort



1) Splitting the data between features and labels

In [6]:
Y = data['labels']
X = data.drop('labels', axis = 1)

2) Splitting between training and test set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
#keith, here we can just stratify but i dont know how cause there is no variable label in the x
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, train_size = 0.8)


Defining the model and hyperparam

In [ ]:
#number of trees
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num =40)]
#min number of sample in each leaf
min_samples_split = [int(x) for x in np.linspace(15,25,10)]
#max number level of questions in each tree
max_depth = [int(x) for x in np.linspace(2,5,3)]

In [ ]:
#hyperparam tuning
param_grid = {
    'n_estimators' : n_estimators,
    'min_samples_split': min_samples_split,
    'max_depth': max_depth
}
param_grid


I will now define the model so that we can apply that hyperparam grid

In [ ]:
model = RandomForestClassifier()

In [ ]:
#importing the grid that allows us to tune
from sklearn.model_selection import GridSearchCV

#applying the grid i created
rf_grid = GridSearchCV(model, param_grid, cv = 3, verbose = 2)
#{'max_depth': 5, 'min_samples_split': 22, 'n_estimators': 30} best one, 

Now before fitting, Id like to define all the possibility of different hyperparameters

In [ ]:
rf_grid.fit(x_train,y_train)

In [ ]:
rf_grid.best_params_

{'max_depth': 5, 'min_samples_split': 22, 'n_estimators': 30}

In [ ]:
rf_grid.score(x_train, y_train)

0.9151515151515152

In [ ]:
rf_grid.score(x_test, y_test)

0.927710843373494

best hyperparams: {'max_depth': 5, 'min_samples_split': 22, 'n_estimators': 30}
the score for training was 0.91
the score for test was 0.92

gonna run a confusion matrix now